In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm.notebook import tqdm
import argparse
import pickle
import random
from utils import PklsFolder
from model import C_LSTM

def normalize(x):
    return (x / 255) -0.5

def denormalize(x):
    return (x + 0.5) * 255

In [2]:
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

train_set_idx = np.load('')
flow_dataset = PklsFolder('')

In [3]:
len(train_set_idx)==59877, len(train_set_idx)

(True, 59877)

In [8]:
#training param
batch_size = 128
num_epoch = 150 # In the paper, this value is 1.

train_random_sampler = torch.utils.data.SubsetRandomSampler(train_set_idx)
train_dataloader = DataLoader(flow_dataset, batch_size = batch_size, shuffle = False,  sampler = train_random_sampler)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
model = C_LSTM().to(device)
criterion = torch.nn.BCELoss()  #BCELoss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [11]:
model.train()
model_save_path = './weight_train_2012/'

if not os.path.isdir(model_save_path):                                                           
    os.mkdir(model_save_path)

loss_history = []
for epoch in range(num_epoch):
    epoch_loss = 0
    for batch_id, (train_x, train_y, _) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        train_x = normalize(train_x)
        preds = model(train_x.float().to(device))
        loss = criterion(preds.reshape(-1), train_y.float().to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        if (batch_id+1)% 100 == 0:
            print(f' iter : {batch_id+1} avg_loss : {epoch_loss/(batch_id+1)}')
            
    loss_history.append(epoch_loss)
    print(f'epoch : {epoch+1}, total train loss : {epoch_loss}')
    with open(f'{model_save_path}model_epoch_{epoch+1}.pt','wb') as f:
        torch.save({
            'state': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch,
            'loss_history': loss_history,
        }, f)

print('Train finished.')

  0%|          | 0/468 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


 iter : 100 avg_loss : 0.4535064724087715
 iter : 200 avg_loss : 0.3319469366967678
 iter : 300 avg_loss : 0.26296752590686084
 iter : 400 avg_loss : 0.22208439881913364
epoch : 1, total train loss : 94.78982685692608


  0%|          | 0/468 [00:00<?, ?it/s]

 iter : 100 avg_loss : 0.07384871578775347
 iter : 200 avg_loss : 0.06750886833760887
 iter : 300 avg_loss : 0.06374282008036972
 iter : 400 avg_loss : 0.06219333733897656
epoch : 2, total train loss : 27.848806705325842


...

 iter : 100 avg_loss : 0.00039769898943063707
 iter : 200 avg_loss : 0.00043898285217522923
 iter : 300 avg_loss : 0.0004631337761755579
 iter : 400 avg_loss : 0.0003704698072254331
epoch : 148, total train loss : 0.20154901650309398


  0%|          | 0/468 [00:00<?, ?it/s]

 iter : 100 avg_loss : 9.328969730773906e-05
 iter : 200 avg_loss : 0.0004496960959735929
 iter : 300 avg_loss : 0.00046146923683627016
 iter : 400 avg_loss : 0.00038857670411175606
epoch : 149, total train loss : 0.4774215957090746


  0%|          | 0/468 [00:00<?, ?it/s]

 iter : 100 avg_loss : 0.0023315143496256494
 iter : 200 avg_loss : 0.0017471480588847044
 iter : 300 avg_loss : 0.001371300665361256
 iter : 400 avg_loss : 0.0013035568622137817
epoch : 150, total train loss : 0.5520148157675067
